In [2]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

/var/folders/__/51nntbcd363626dtly5tl0r00000gn/T/ipykernel_59010/767406085.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [294]:
from cobra.io import read_sbml_model
model = read_sbml_model('iNF517.xml')
medium = model.medium
model.medium = medium

In [3]:
len(model.metabolites)

650

In [214]:
# target metabolite
model.metabolites.mqn7_c

Metabolite identifier,mqn7_c
Name,Menaquinone 7 C46H64O2
Memory address,0x1647104d0
Formula,C46H64O2
Compartment,c
In 3 reaction(s),"CYTB_B2, NADH4, G3PD4"


Starting from the end, the reaction where mqn7_c present, I was checking the steps up towards the reactions that I'm checking below the part with mqn. These reactions end with dhna_c.

In [215]:
# CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
for reaction in model.metabolites.mqn7_c.reactions:
    print(reaction, reaction.name)

CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
NADH4: h_c + mqn7_c + nadh_c --> mql7_c + nad_c NADH dehydrogenase  Menaquinone 7  no proton
G3PD4: glyc3p_c + mqn7_c --> dhap_c + mql7_c Glycerol 3 phosphate dehydrogenase  menaquinone 7


In [216]:
model.reactions.CYTB_B2.bounds

(0.0, 0.0)

In [234]:
reaction_name = "CYTB_B2"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = 0
new_upper_bound = 1000
reaction.lower_bound = new_lower_bound
reaction.upper_bound = new_upper_bound
reaction.bounds

(0, 1000)

In [235]:
for reaction in model.metabolites.mql7_c.reactions:
    print(reaction, reaction.name)

G3PD4: glyc3p_c + mqn7_c --> dhap_c + mql7_c Glycerol 3 phosphate dehydrogenase  menaquinone 7
CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
NADH4: h_c + mqn7_c + nadh_c --> mql7_c + nad_c NADH dehydrogenase  Menaquinone 7  no proton


In [283]:
# want to repeat this reaction, but there is no demethylmenaquinol 7
# https://www.kegg.jp/entry/R09736

In [295]:
# amet_c + 2dmmq7_c ⇌ ahcys_c + h_c + mqn7_c
# http://bigg.ucsd.edu/universal/reactions/MQNS

new_reaction2 = Reaction('MQNS') # 4-Hydroxyphenylpyruvateoxygen oxidoreductase
demethylmenaquinol7 = Metabolite(id='2dmmq7_c', compartment='c')

new_reaction2.add_metabolites({model.metabolites.get_by_id('amet_c'): -1,
                               demethylmenaquinol7: -1,
                               model.metabolites.get_by_id('ahcys_c'): 1,
                               model.metabolites.get_by_id('h_c'): 1,
                               model.metabolites.get_by_id('mqn7_c'): 1
                              })

model.add_reactions([new_reaction2])

In [237]:
# checking that the reaction was added
for reaction in model.metabolites.mqn7_c.reactions:
    print(reaction, reaction.name)

G3PD4: glyc3p_c + mqn7_c --> dhap_c + mql7_c Glycerol 3 phosphate dehydrogenase  menaquinone 7
MQNS: 2dmmq7_c + amet_c --> ahcys_c + h_c + mqn7_c 
CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
NADH4: h_c + mqn7_c + nadh_c --> mql7_c + nad_c NADH dehydrogenase  Menaquinone 7  no proton


In [344]:
reaction_name = "MQNS"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = 0.03
reaction.lower_bound = new_lower_bound
model.reactions.MQNS.bounds

(0.03, 1000.0)

In [345]:
# adding this reaction to obtain 2dmmq7_c from somewhere
# not found in L. lactis though
# http://bigg.ucsd.edu/universal/reactions/DHNAOT7
# dhna_c + nad_c + hepdp_c ⇌ co2_c + nadh_c + ppi_c + 2dmmq7_c
new_reaction6 = Reaction('DHNAOT7') 

new_reaction6.add_metabolites({model.metabolites.get_by_id('dhna_c'): -1,
                               model.metabolites.get_by_id('nad_c'): -1,
                               model.metabolites.get_by_id('hepdp_c'): -1,
                               model.metabolites.get_by_id('co2_c'): 1,
                               model.metabolites.get_by_id('nadh_c'): 1,
                               model.metabolites.get_by_id('ppi_c'): 1,
                               model.metabolites.get_by_id('2dmmq7_c'): 1
                              })

model.add_reactions([new_reaction6])

Ignoring reaction 'DHNAOT7' since it already exists.


In [346]:
model.objective = model.reactions.DM_mqn7_c
production_mqn7c = model.optimize().objective_value
production_mqn7c

0.04099999999999983

In [239]:
reaction_name = "CYTB_B2"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = 0
reaction.lower_bound = new_lower_bound
model.reactions.CYTB_B2.bounds

(0, 1000)

In [223]:
model.reactions.MQNS

Reaction identifier,MQNS
Name,
Memory address,0x177741850
Stoichiometry,2dmmq7_c + amet_c --> ahcys_c + h_c + mqn7_c + S-Adenosyl-L-methionine --> S-Adenosyl-L-homocysteine + H+ + Menaquinone 7 C46H64O2
GPR,
Lower bound,0.001
Upper bound,1000.0


When the lower boundary of CYTB_B2 is set for higher than 0 (have tried till 0.00001), the process becomes infeasible. So, it's better to try adjusting MQNS.

In [269]:
model.objective = model.reactions.DM_mqn7_c
selected_genes = set()

for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()
    if (mutant.slim_optimize(error_value=0.) > 0.001):
        selected_genes.add(gene.id)

In [270]:
selected_genes

set()

In [301]:
# attempt with that reaction
# http://bigg.ucsd.edu/universal/reactions/POX3
# h2o_c + pyr_c + mqn7_c ⇌ ac_c + co2_c + mql7_c
new_reaction3 = Reaction('POX3') 

new_reaction3.add_metabolites({model.metabolites.get_by_id('ac_c'): -1,
                               model.metabolites.get_by_id('co2_c'): -1,
                               model.metabolites.get_by_id('mql7_c'): -1,
                               model.metabolites.get_by_id('h2o_c'): 1,
                               model.metabolites.get_by_id('pyr_c'): 1,
                               model.metabolites.get_by_id('mqn7_c'): 1
                              })

model.add_reactions([new_reaction3])

In [243]:
for reaction in model.metabolites.mqn7_c.reactions:
    print(reaction, reaction.name)

POX3: ac_c + co2_c + mql7_c --> h2o_c + mqn7_c + pyr_c 
MQNS: 2dmmq7_c + amet_c --> ahcys_c + h_c + mqn7_c 
NADH4: h_c + mqn7_c + nadh_c --> mql7_c + nad_c NADH dehydrogenase  Menaquinone 7  no proton
CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
G3PD4: glyc3p_c + mqn7_c --> dhap_c + mql7_c Glycerol 3 phosphate dehydrogenase  menaquinone 7
DM_mqn7_c: mqn7_c -->  Menaquinone 7 C46H64O2 demand


In [302]:
reaction_name = "POX3"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = 0.001
reaction.lower_bound = new_lower_bound
model.reactions.POX3.bounds

(0.001, 1000.0)

In [312]:
model.objective = model.reactions.DM_mqn7_c
production_mqn7c = model.optimize().objective_value
production_mqn7c

/Users/kseniakirdey/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


0.0

In [303]:
# http://bigg.ucsd.edu/universal/reactions/AMMQT7
# trying to add 2dmmq7_c
# amet_c + nadph_c + 2dmmq7_c ⇌ ahcys_c + nadp_c + mql7_c

new_reaction4 = Reaction('AMMQT7') 

new_reaction4.add_metabolites({model.metabolites.get_by_id('amet_c'): 1,
                               model.metabolites.get_by_id('nadph_c'): 1,
                               model.metabolites.get_by_id('2dmmq7_c'): 1,
                               model.metabolites.get_by_id('ahcys_c'): -1,
                               model.metabolites.get_by_id('nadp_c'): -1,
                               model.metabolites.get_by_id('mql7_c'): -1
                              })

model.add_reactions([new_reaction4])

In [313]:
# https://www.kegg.jp/entry/R09445
# want to add this reaction, but there're no such enzymes in BIGG
# fadh2_c + mqn7_c ⇌ fad_c + mql7_c

new_reaction5 = Reaction('FADMQOR') 

new_reaction5.add_metabolites({model.metabolites.get_by_id('mql7_c'): -1,
                               model.metabolites.get_by_id('fad_c'): -1,
                               model.metabolites.get_by_id('mqn7_c'): 1,
                               model.metabolites.get_by_id('fadh2_c'): 1
                              })

model.add_reactions([new_reaction5])

In [272]:
for reaction in model.metabolites.mqn7_c.reactions:
    print(reaction, reaction.name)

POX3: ac_c + co2_c + mql7_c --> h2o_c + mqn7_c + pyr_c 
FADMQOR: fad_c + mql7_c --> fadh2_c + mqn7_c 
MQNS: 2dmmq7_c + amet_c --> ahcys_c + h_c + mqn7_c 
NADH4: h_c + mqn7_c + nadh_c --> mql7_c + nad_c NADH dehydrogenase  Menaquinone 7  no proton
CYTB_B2: 2.0 h_c + mql7_c + 0.5 o2_c --> h2o_c + 2.0 h_e + mqn7_c Menaquinol oxidase  71 protons
G3PD4: glyc3p_c + mqn7_c --> dhap_c + mql7_c Glycerol 3 phosphate dehydrogenase  menaquinone 7
DM_mqn7_c: mqn7_c -->  Menaquinone 7 C46H64O2 demand


In [321]:
# also doesn't work when adjusting the lower bound
reaction_name = "FADMQOR"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = 0
reaction.lower_bound = new_lower_bound
model.reactions.FADMQOR.bounds

(0, 1000.0)

In [322]:
# so far, optimization worked only with POX3
model.objective = model.reactions.DM_mqn7_c
production_mqn7c = model.optimize().objective_value
production_mqn7c

0.001

In [11]:
# CHECKING METABOLITES HERE
for metabolite in model.metabolites:
    if "chor" in metabolite.id:
        print(metabolite.id)

chor_c
ichor_c


> The pathway that starts with chorismate <-> isochorismate. 

In [12]:
# first reaction: Chorismate <=> Isochorismate
# enzyme: menF
for reaction in model.metabolites.chor_c.reactions:
    print(reaction, reaction.name)

CHORS: 3psme_c --> chor_c + pi_c Chorismate synthase
ICHORS: chor_c <=> ichor_c Isochorismate synthase
ADCS: chor_c + gln__L_c --> 4adcho_c + glu__L_c 4-amino-4-deoxychorismate synthase
CHORM: chor_c --> pphn_c Chorismate mutase
ANS: chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c Anthranilate synthase


In [14]:
reaction = model.reactions.ICHORS  # Replace with the ID of the reaction you want to check
print(reaction.bounds)

(-1000.0, 1000.0)


In [34]:
# we can increase the lower bound, so less isochorismate will be converted back into chorismate
ichors_reaction = model.reactions.ICHORS

new_lower_bound = -100
ichors_reaction.lower_bound = new_lower_bound

print(model.reactions.ICHORS.bounds)

(-100, 1000.0)


---

In [35]:
# second reaction: sochorismate + 2-Oxoglutarate <=> 2-Succinyl-5-enolpyruvyl-6-hydroxy-3-cyclohexene-1-carboxylate + CO2
# combined with third reaction of production of (1R,6R)-6-Hydroxy-2-succinylcyclohexa-2,4-diene-1-carboxylate
# enzyme: MenD, MenH
for reaction in model.metabolites.ichor_c.reactions:
    print(reaction, reaction.name)

2S6HCCi: akg_c + h_c + ichor_c <=> 2shchc_c + co2_c + pyr_c 2 succinyl 6 hydroxy 2 4 cyclohexadiene 1 carboxylate synthase
ICHORS: chor_c <=> ichor_c Isochorismate synthase


In [42]:
reaction_name = "2S6HCCi"
reaction = model.reactions.get_by_id(reaction_name)
new_lower_bound = -100
reaction.lower_bound = new_lower_bound
reaction.bounds

(-100, 1000.0)

---

In [49]:
# fourth reaction: production of succinylbenzoate
# goes only one side, so there's no need to adjust the boundary

metabolite_id = "2shchc_c"
metabolite = model.metabolites.get_by_id(metabolite_id)
for reaction in metabolite.reactions:
    print(reaction, reaction.name)

2S6HCCi: akg_c + h_c + ichor_c <=> 2shchc_c + co2_c + pyr_c 2 succinyl 6 hydroxy 2 4 cyclohexadiene 1 carboxylate synthase
SUCBZS: 2shchc_c --> h2o_c + sucbz_c O-succinylbenzoate-CoA synthase


---

In [52]:
# fifth reaction: ATP + 2-Succinylbenzoate + CoA <=> AMP + Diphosphate + 2-Succinylbenzoyl-CoA

for reaction in model.metabolites.sucbz_c.reactions:
    print(reaction, reaction.name)

SUCBZL: atp_c + coa_c + sucbz_c --> amp_c + ppi_c + sbzcoa_c O-succinylbenzoate-CoA ligase
SUCBZS: 2shchc_c <=> h2o_c + sucbz_c O-succinylbenzoate-CoA synthase


In [55]:
# goes only one side, so there's no need to adjust the boundary
print(model.reactions.SUCBZL.bounds)

(0.0, 1000.0)


---

In [57]:
# sixth reaction: 2-Succinylbenzoyl-CoA <=> 1,4-Dihydroxy-2-naphthoyl-CoA + H2O
# there's only the production of 1,4-Dihydroxy-2-naphthoate. So, let's add the reaction of CoA addition.

for reaction in model.metabolites.sbzcoa_c.reactions:
    print(reaction, reaction.name)

SUCBZL: atp_c + coa_c + sucbz_c --> amp_c + ppi_c + sbzcoa_c O-succinylbenzoate-CoA ligase
NPHS: sbzcoa_c --> coa_c + dhna_c Naphthoate synthase


In [90]:
print(model.reactions.NPHS.bounds)

(0.0, 1000.0)


---

All the reactions seem checked with regard to the bounds.

The first reaction is the one that transforms 	
1,4-Dihydroxy-2-naphthoate into 2-Demethylmenaquinol-8 that is needed for the future pathway. Enzyme: menA (DHNAOT_1).
The next reaction has menaquinol as a product. In Lactococcus lactis model, mql7_c was already present, and from the BIGG data, there's no information about mql8_c being present. So, we have to obtain 2dmmq7_c to obtain mql7_c. Even though no information is depicted about 2dmmq7_c neither ...

In [103]:
from cobra import Reaction, Metabolite

# first new reaction
# Demethylmenaquinol + S-Adenosyl-L-methionine <=> Menaquinol + S-Adenosyl-L-homocysteine
# dhna_c + nad_c + octdp_c --> 2dmmq7_c + co2_c + nadh_c + ppi_c

new_reaction1 = Reaction('DHNAOT1') # 4-Hydroxyphenylpyruvateoxygen oxidoreductase
demethylmenaquinol7 = Metabolite(id='2dmmq7_c', compartment='c')

new_reaction1.add_metabolites({model.metabolites.get_by_id('dhna_c'): -1,
                               model.metabolites.get_by_id('nad_c'): -1,
                               model.metabolites.get_by_id('octdp_c'): -1,
                               demethylmenaquinol7: 1,
                               model.metabolites.get_by_id('nadh_c'): 1,
                               model.metabolites.get_by_id('ppi_c'): 1
                              })

model.add_reactions([new_reaction1])

Now, we can obtain the required menaquinol-7. 

In [104]:
# second new reaction
# amet_c + nadph_c + 2dmmq7_c ⇌ ahcys_c + nadp_c + mql7_c 
# http://bigg.ucsd.edu/universal/reactions/AMMQT7
new_reaction2 = Reaction('AMMQT7') # 4-Hydroxyphenylpyruvateoxygen oxidoreductase

new_reaction2.add_metabolites({model.metabolites.get_by_id('amet_c'): -1,
                               model.metabolites.get_by_id('nadph_c'): -1,
                               demethylmenaquinol7: -1,
                               model.metabolites.get_by_id('ahcys_c'): 1,
                               model.metabolites.get_by_id('nadp_c'): 1,
                               model.metabolites.get_by_id('mql7_c'): 1
                              })

model.add_reactions([new_reaction2])

Looking at the reactions of mql7_c, we can see that there's one of our interest - with menaquinol oxidase.

In order to simulate the production of alpha-Tocopherol with the model, a boundary reaction of the type 'demand' is added.

In [68]:
model.add_boundary(model.metabolites.mqn7_c, type='demand')
model.objective = model.reactions.DM_mqn7_c
production_mqn7c = model.optimize().objective_value
production_mqn7c

0.0

Another trial

In [ ]:
from cobra.io import read_sbml_model
model = read_sbml_model('iNF517.xml')
medium = model.medium
model.medium = medium

In [111]:
#
model.objective = model.reactions.DM_mqn7_c

In [115]:
selected_genes = set()

for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()
    if (mutant.slim_optimize(error_value=0.) < 0.05):
        selected_genes.add(gene.id)

KeyboardInterrupt: 

In [114]:
selected_genes

set()